In [1]:
// place a csv file named simple_data.csv in the same folder as this notebook 
// file contents:
//
//id,data
//100, {'name':'037717956'}
//200, {'name':'077564444'}

org.apache.spark.sql.SparkSession@52f04e96

In [59]:
case class SimpleData(name: String)
val simpleJsonStruct = Encoders.product[SimpleData].schema
simpleJsonStruct

StructType(StructField(name,StringType,true))

In [64]:
val simpleDF = spark.read.option("header","true").csv("simple_data.csv")

In [65]:
simpleDF.printSchema()
simpleDF.show()

root
 |-- id: string (nullable = true)
 |-- data: string (nullable = true)

+---+--------------------+
| id|                data|
+---+--------------------+
|100| {'name':'0377179...|
|200| {'name':'0775644...|
+---+--------------------+



In [69]:
val parsedDF = simpleDF.withColumn("data2", from_json(col("data"), simpleJsonStruct))
parsedDF.show()

+---+--------------------+-----------+
| id|                data|      data2|
+---+--------------------+-----------+
|100| {'name':'0377179...|[037717956]|
|200| {'name':'0775644...|[077564444]|
+---+--------------------+-----------+



In [70]:
parsedDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- data: string (nullable = true)
 |-- data2: struct (nullable = true)
 |    |-- name: string (nullable = true)



In [72]:
parsedDF.select("data2.name").show()

+---------+
|     name|
+---------+
|037717956|
|077564444|
+---------+

